In [1]:
import numpy as np
import re
import string
import csv
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import pickle as pkl
import gensim

In [2]:
from GetData import getData

In [3]:
data_rows,X,Y=getData()

In [8]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
glove_input_file = "../data/glove.twitter.27B.200d.txt"
word2vec_output_file = "word2vec.txt"
glove2word2vec(glove_input_file, word2vec_output_file)
Model= KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [96]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

# convert text data into term-frequency matrix
data = cv.fit_transform(X)
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='utf-8', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(X).toarray()    

word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))



In [107]:
def getSentenceVectors(X):
    sentenceVectors=list()
    sentenceVectors=np.zeros(shape=(len(X),200))
    i=0
    for text in X:
        text_vector=list()
        for word in text.split():
            if word in Model.vocab:
                text_vector.append(Model[word])
            else:
                text_vector.append(np.zeros(shape=(1,200)))
        text_vector=np.mean(text_vector,axis=0)
        #text_vector = np.mean([Model[word]*word2tfidf.get(word,0) for word in text.split() if word in Model.vocab], axis=0)
        sentenceVectors[i]=text_vector
        i=i+1
    return sentenceVectors

In [108]:
X_vectors=getSentenceVectors(X)

In [109]:
X_vectors=np.asarray(X_vectors)

In [100]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
print(cross_val_score(clf, X_vectors, Y, cv=10))  

ValueError: Input X must be non-negative

In [110]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-4,early_stopping=True,hidden_layer_sizes=(50), random_state=1)
print(cross_val_score(clf, X_vectors, Y, cv=10)) 

[0.44736842 0.48684211 0.49333333 0.60273973 0.51388889 0.54929577
 0.66197183 0.70422535 0.66197183 0.5915493 ]


In [111]:
from sklearn.svm import SVC
clf =  SVC(kernel='linear')
print(cross_val_score(clf, X_vectors, Y, cv=10)) 

[0.51315789 0.56578947 0.46666667 0.63013699 0.63888889 0.56338028
 0.70422535 0.70422535 0.67605634 0.5915493 ]


In [113]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)
print(cross_val_score(clf, X_vectors, Y, cv=10)) 

[0.5        0.57894737 0.45333333 0.63013699 0.54166667 0.61971831
 0.5915493  0.63380282 0.66197183 0.63380282]
